In [4]:
#from bertviz.transformers_neuron_view import BertModel
#from bertviz.neuron_view import show
from transformers import AutoTokenizer

In [6]:
model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
#model = BertModel.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [7]:
text = 'time flies like an arrow'
#show(
#    model, 
#    'bert',
#    tokenizer,
#    text,
#    display_mode='light',
#    layer=0,
#    head=8)